Завдання до лабораторної роботи
Створити програму, яка зчитує заданий набір даних з попередньої
лабораторної роботи, виділяє текстову частину даних (вони розглядаються
як документи), виконує попередню обробку та завдання відповідно до
варіанту.
Варіант 1.
1.Застосувати приховане семантичне індексування бібліотеки Gensim
для моделювання тем. Вивести документи, що зробили найбільший
вклад в теми. Обрати три нових документи та визначити їх теми.
2. Використати текст austen-persuasion.txt з корпусу gutenberg
бібліотеки nltk та вивести ключові біграми.

In [1]:
import pandas as pd
import nltk
import re
import numpy as np
import gensim
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords, gutenberg
from operator import itemgetter

In [2]:
# імпортуємо таблицю з нестандартним роздільником
data = pd.read_csv("bbc-news-data.csv", header=0, sep='\t')
column_names = data.columns.tolist()

print(data.head(5))

   category filename                              title   
0  business  001.txt  Ad sales boost Time Warner profit  \
1  business  002.txt   Dollar gains on Greenspan speech   
2  business  003.txt  Yukos unit buyer faces loan claim   
3  business  004.txt  High fuel prices hit BA's profits   
4  business  005.txt  Pernod takeover talk lifts Domecq   

                                             content  
0   Quarterly profits at US media giant TimeWarne...  
1   The dollar has hit its highest level against ...  
2   The owners of embattled Russian oil giant Yuk...  
3   British Airways has blamed high fuel prices f...  
4   Shares in UK drinks and food firm Allied Dome...  


In [3]:
# за допомогою функції з поперідньої лабораторної обробляємо наш текст
wpt = nltk.WordPunctTokenizer()
stop_words = nltk.corpus.stopwords.words('english')

def preproc_doc(doc):
    doc = re.sub(r'[^a-zA-Z\s]', '', doc)
    doc = doc.lower().strip()
    tokens = word_tokenize(doc)
    stop_words = set(stopwords.words('english'))
    filtered_tokens = [token for token in tokens if token not in stop_words]
    doc = ' '.join(filtered_tokens)
    return doc

data['content'] = data['content'].apply(preproc_doc)
    
print(data.head(5))

   category filename                              title   
0  business  001.txt  Ad sales boost Time Warner profit  \
1  business  002.txt   Dollar gains on Greenspan speech   
2  business  003.txt  Yukos unit buyer faces loan claim   
3  business  004.txt  High fuel prices hit BA's profits   
4  business  005.txt  Pernod takeover talk lifts Domecq   

                                             content  
0  quarterly profits us media giant timewarner ju...  
1  dollar hit highest level euro almost three mon...  
2  owners embattled russian oil giant yukos ask b...  
3  british airways blamed high fuel prices drop p...  
4  shares uk drinks food firm allied domecq risen...  


In [4]:
# виконуємо пошук пустих документі, які могли виникнути під час обробки
data = data.replace(r'^(\s?)+$', np.nan, regex=True)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2225 entries, 0 to 2224
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   category  2225 non-null   object
 1   filename  2225 non-null   object
 2   title     2225 non-null   object
 3   content   2225 non-null   object
dtypes: object(4)
memory usage: 69.7+ KB


In [5]:
# видаляємо ці документи
data = data.dropna().reset_index(drop=True)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2225 entries, 0 to 2224
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   category  2225 non-null   object
 1   filename  2225 non-null   object
 2   title     2225 non-null   object
 3   content   2225 non-null   object
dtypes: object(4)
memory usage: 69.7+ KB


In [6]:
# виділяємо текстову частину даних для подальшої обробки
text_data = data['content']
tokenized_data = [word_tokenize(text) for text in text_data]

In [7]:
# витягуємо з тексту деякі корисні фрази на основі біграм та видаляємо непотрібні терміни
bigram = gensim.models.Phrases(tokenized_data, min_count=20, threshold=20, delimiter='_')
bigram_model = gensim.models.phrases.Phraser(bigram)
# виділяємо біграми для всих документів
text_bigrams = [bigram_model[doc] for doc in tokenized_data]
print(text_bigrams[:1])


[['quarterly', 'profits', 'us', 'media', 'giant', 'timewarner', 'jumped', 'bn', 'three_months', 'december', 'yearearlier', 'firm', 'one', 'biggest', 'investors', 'google', 'benefited', 'sales', 'highspeed', 'internet', 'connections', 'higher', 'advert', 'sales', 'timewarner', 'said', 'fourth_quarter', 'sales', 'rose', 'bn_bn', 'profits', 'buoyed', 'oneoff', 'gains', 'offset', 'profit', 'dip', 'warner', 'bros', 'less', 'users', 'aol', 'time', 'warner', 'said', 'friday', 'owns', 'searchengine', 'google', 'internet', 'business', 'aol', 'mixed', 'fortunes', 'lost', 'subscribers', 'fourth_quarter', 'profits', 'lower', 'preceding', 'three', 'quarters', 'however', 'company', 'said', 'aols', 'underlying', 'profit', 'exceptional', 'items', 'rose', 'back', 'stronger', 'internet', 'advertising', 'revenues', 'hopes', 'increase', 'subscribers', 'offering', 'online', 'service', 'free', 'timewarner', 'internet', 'customers', 'try', 'sign', 'aols', 'existing', 'customers', 'highspeed', 'broadband', 't

In [8]:
# створюємо словник
dictionary = gensim.corpora.Dictionary(text_bigrams)
# форматуємо ключі та значення словника в рядок
output = ', '.join(f"('{token_id}', '{token}')" for token, token_id in dictionary.token2id.items())
print(f"Розмір словника: {len(dictionary)}\n{output}>")

Розмір словника: 31469
('0', 'accounts'), ('1', 'adjust'), ('2', 'advert'), ('3', 'advertising'), ('4', 'alexander'), ('5', 'already'), ('6', 'also'), ('7', 'amount'), ('8', 'analysts'), ('9', 'aol'), ('10', 'aols'), ('11', 'around'), ('12', 'aside'), ('13', 'back'), ('14', 'benefited'), ('15', 'bertelsmanns'), ('16', 'better'), ('17', 'biggest'), ('18', 'bn'), ('19', 'bn_bn'), ('20', 'book'), ('21', 'boosted'), ('22', 'boxoffice'), ('23', 'broadband'), ('24', 'bros'), ('25', 'buoyed'), ('26', 'business'), ('27', 'catwoman'), ('28', 'chairman'), ('29', 'charges'), ('30', 'chief_executive'), ('31', 'close'), ('32', 'commission'), ('33', 'company'), ('34', 'concluding'), ('35', 'connections'), ('36', 'contrast'), ('37', 'customers'), ('38', 'deal'), ('39', 'december'), ('40', 'dip'), ('41', 'division'), ('42', 'earnings'), ('43', 'efforts'), ('44', 'enhancing'), ('45', 'estimate'), ('46', 'europe'), ('47', 'exceeding'), ('48', 'exceptional'), ('49', 'existing'), ('50', 'expectations'), (

In [9]:
# зменшуємо об'єм словника за рахунок видалення слів, які зустрічаються менше 20 разів, 
# а також слова, які зустрічються в більше ніж 60 % документів
dictionary.filter_extremes(no_below=20, no_above=0.6)
output = ', '.join(f"('{token_id}', '{token}')" for token, token_id in dictionary.token2id.items())
print(f"Розмір словника: {len(dictionary)}\n{output}>")

Розмір словника: 3360
('0', 'accounts'), ('1', 'advertising'), ('2', 'alexander'), ('3', 'already'), ('4', 'also'), ('5', 'amount'), ('6', 'analysts'), ('7', 'around'), ('8', 'aside'), ('9', 'back'), ('10', 'better'), ('11', 'biggest'), ('12', 'bn'), ('13', 'bn_bn'), ('14', 'book'), ('15', 'boosted'), ('16', 'broadband'), ('17', 'business'), ('18', 'chairman'), ('19', 'charges'), ('20', 'chief_executive'), ('21', 'close'), ('22', 'commission'), ('23', 'company'), ('24', 'connections'), ('25', 'contrast'), ('26', 'customers'), ('27', 'deal'), ('28', 'december'), ('29', 'division'), ('30', 'earnings'), ('31', 'efforts'), ('32', 'estimate'), ('33', 'europe'), ('34', 'existing'), ('35', 'expectations'), ('36', 'expects'), ('37', 'film'), ('38', 'final'), ('39', 'financial'), ('40', 'firm'), ('41', 'flexibility'), ('42', 'following'), ('43', 'fourth_quarter'), ('44', 'free'), ('45', 'friday'), ('46', 'gains'), ('47', 'german'), ('48', 'giant'), ('49', 'google'), ('50', 'grew'), ('51', 'grow

In [10]:
# створюємо модель "Сумка слів"
bow_corpus = [dictionary.doc2bow(text) for text in text_bigrams]
print(bow_corpus)

[[(0, 2), (1, 2), (2, 1), (3, 1), (4, 2), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 3), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 1), (21, 1), (22, 1), (23, 2), (24, 1), (25, 1), (26, 2), (27, 2), (28, 1), (29, 1), (30, 1), (31, 1), (32, 1), (33, 2), (34, 1), (35, 1), (36, 1), (37, 2), (38, 1), (39, 1), (40, 1), (41, 1), (42, 1), (43, 3), (44, 1), (45, 1), (46, 1), (47, 1), (48, 1), (49, 2), (50, 1), (51, 1), (52, 1), (53, 2), (54, 2), (55, 1), (56, 1), (57, 1), (58, 1), (59, 4), (60, 1), (61, 1), (62, 1), (63, 1), (64, 1), (65, 1), (66, 1), (67, 1), (68, 1), (69, 1), (70, 1), (71, 1), (72, 1), (73, 1), (74, 1), (75, 1), (76, 1), (77, 1), (78, 1), (79, 1), (80, 1), (81, 1), (82, 1), (83, 1), (84, 2), (85, 1), (86, 1), (87, 4), (88, 5), (89, 1), (90, 1), (91, 1), (92, 1), (93, 1), (94, 1), (95, 2), (96, 2), (97, 2), (98, 1), (99, 1), (100, 2), (101, 1), (102, 3), (103, 1), (104, 1), (105, 1), (106, 2), (107, 1), (108, 1), (109, 1), (110, 1)

In [11]:
# реалізовуємо модель прихованого семантичного індексування
Total_topics = 10
lsi_bow = gensim.models.LsiModel(bow_corpus, id2word=dictionary, num_topics=Total_topics, onepass=True, chunksize=1740, power_iters=1000)
# виводимо основні теми
for topic_id, topic in lsi_bow.print_topics(num_topics=10, num_words=20):
    print('Topic #'+str(topic_id+1)+':')
    print(topic)

Topic #1:
0.266*"would" + 0.253*"people" + 0.197*"mr" + 0.184*"also" + 0.171*"one" + 0.170*"new" + 0.161*"us" + 0.153*"could" + 0.126*"music" + 0.124*"government" + 0.116*"like" + 0.107*"time" + 0.101*"get" + 0.096*"many" + 0.093*"first" + 0.091*"make" + 0.089*"year" + 0.085*"uk" + 0.085*"two" + 0.083*"game"
Topic #2:
-0.455*"music" + 0.248*"mr" + -0.219*"best" + 0.206*"would" + 0.193*"government" + -0.153*"game" + -0.146*"song" + 0.132*"labour" + -0.118*"awards" + -0.098*"games" + -0.095*"win" + -0.094*"award" + -0.093*"good" + -0.090*"im" + -0.089*"like" + -0.085*"film" + 0.083*"bn" + -0.083*"black" + -0.081*"play" + -0.081*"think"
Topic #3:
-0.470*"music" + 0.286*"game" + -0.254*"people" + 0.154*"games" + 0.152*"england" + 0.140*"wales" + 0.133*"first" + 0.113*"win" + 0.109*"two" + 0.103*"time" + 0.100*"play" + 0.097*"back" + -0.094*"technology" + 0.093*"ireland" + 0.092*"side" + 0.084*"world" + -0.081*"services" + 0.080*"team" + -0.078*"mobile" + -0.078*"users"
Topic #4:
-0.211*"mu

In [12]:
# розділяємо кожну тему на дві підтеми на основі напрямку векторів
for n in range(Total_topics):
    print('Topic #'+str(n+1)+':')
    print('='*50)
    d1 = []
    d2 = []
    for term, wt in lsi_bow.show_topic(n, topn=20):
        if wt >= 0:
            d1.append((term, round(wt, 3)))
        else:
            d2.append((term, round(wt, 3)))
            
    print("d1:", d1)
    print("d2:", d2)

Topic #1:
d1: [('would', 0.266), ('people', 0.253), ('mr', 0.197), ('also', 0.184), ('one', 0.171), ('new', 0.17), ('us', 0.161), ('could', 0.153), ('music', 0.126), ('government', 0.124), ('like', 0.116), ('time', 0.107), ('get', 0.101), ('many', 0.096), ('first', 0.093), ('make', 0.091), ('year', 0.089), ('uk', 0.085), ('two', 0.085), ('game', 0.083)]
d2: []
Topic #2:
d1: [('mr', 0.248), ('would', 0.206), ('government', 0.193), ('labour', 0.132), ('bn', 0.083)]
d2: [('music', -0.455), ('best', -0.219), ('game', -0.153), ('song', -0.146), ('awards', -0.118), ('games', -0.098), ('win', -0.095), ('award', -0.094), ('good', -0.093), ('im', -0.09), ('like', -0.089), ('film', -0.085), ('black', -0.083), ('play', -0.081), ('think', -0.081)]
Topic #3:
d1: [('game', 0.286), ('games', 0.154), ('england', 0.152), ('wales', 0.14), ('first', 0.133), ('win', 0.113), ('two', 0.109), ('time', 0.103), ('play', 0.1), ('back', 0.097), ('ireland', 0.093), ('side', 0.092), ('world', 0.084), ('team', 0.08

In [13]:
# отримуємо матриці, що використовувались для сингулярного розладу
term_topic = lsi_bow.projection.u
singular_values = lsi_bow.projection.s
topic_document = (gensim.matutils.corpus2dense(lsi_bow[bow_corpus], len(singular_values)).T / singular_values).T
print(term_topic, singular_values, topic_document)

[[ 8.18859706e-03  7.69689600e-03  4.43685698e-04 ...  1.52231945e-02
   8.22812571e-03  1.15796028e-02]
 [ 3.56041748e-03  1.14011119e-03 -3.88461163e-03 ...  4.05299188e-06
  -2.10956146e-04  2.59177418e-03]
 [ 2.55736178e-03 -1.48910599e-03  1.70850089e-03 ... -1.11564168e-02
   6.13525076e-03  3.04199696e-03]
 ...
 [ 1.33812658e-02 -1.81009820e-02 -4.08195971e-02 ... -2.78262239e-02
  -1.08745452e-02 -5.07430233e-02]
 [ 7.17722211e-03  7.87839855e-03 -1.35832070e-02 ...  7.68138440e-03
  -3.44398653e-02  3.68484317e-02]
 [ 4.16093668e-03 -6.43060736e-03  9.22992025e-03 ...  5.51809654e-03
   2.71339766e-02 -1.19608282e-02]] [260.0558615  121.88533737 111.3860546  106.04908995  97.76287651
  89.46339568  83.63724403  83.41970688  78.22478619  74.2037986 ] [[ 0.01516765  0.01880561  0.008394   ...  0.04381965  0.02359114
   0.15717702]
 [ 0.00254448  0.01370652  0.01218987 ...  0.01339365  0.02078013
  -0.14954924]
 [-0.00145971  0.00696233  0.00236134 ... -0.00874784 -0.0093213
   0

In [14]:
# складаємо датафрейм, що містить теми документів
document_topics = pd.DataFrame(np.round(topic_document.T, 3), columns=['T'+str(i) for i in range(1, Total_topics+1)])
print(document_topics)

         T1     T2     T3     T4     T5     T6     T7     T8     T9    T10
0     0.015  0.003 -0.001  0.015  0.048  0.021  0.004 -0.008  0.005  0.002
1     0.019  0.014  0.007  0.006  0.053  0.024  0.022  0.021  0.001 -0.007
2     0.008  0.012  0.002 -0.002  0.029  0.018  0.001  0.014  0.016  0.024
3     0.013  0.009  0.006  0.004  0.032  0.024  0.002 -0.006  0.006 -0.008
4     0.007  0.003  0.004  0.004  0.022  0.008 -0.003  0.005 -0.001  0.004
...     ...    ...    ...    ...    ...    ...    ...    ...    ...    ...
2220  0.019  0.007 -0.024  0.035 -0.007  0.004  0.004 -0.022 -0.040  0.023
2221  0.016  0.005 -0.021  0.035 -0.010 -0.006  0.018 -0.007 -0.030  0.035
2222  0.044  0.013 -0.009  0.020  0.002 -0.000  0.015  0.014 -0.029  0.058
2223  0.024  0.021 -0.009  0.015  0.033 -0.027  0.015  0.047 -0.022  0.011
2224  0.157 -0.150  0.176  0.154 -0.283  0.035  0.191  0.078  0.492 -0.022

[2225 rows x 10 columns]


In [15]:
# виводимо документи, що зробили найбільший вклад
top_documents_topic1 = document_topics.sort_values(by='T1', ascending=False).head(5)
print(top_documents_topic1)

         T1     T2     T3     T4     T5     T6     T7     T8     T9    T10
1185  0.225  0.114 -0.082 -0.140 -0.121 -0.246  0.353  0.353 -0.130 -0.109
1275  0.173  0.084 -0.036 -0.037 -0.097  0.262  0.478 -0.201  0.104  0.092
762   0.162 -0.412 -0.159 -0.331  0.071 -0.037  0.232  0.025 -0.045  0.096
2224  0.157 -0.150  0.176  0.154 -0.283  0.035  0.191  0.078  0.492 -0.022
765   0.133 -0.384 -0.367 -0.232 -0.085  0.217 -0.175  0.147  0.024 -0.040


In [17]:
new_documents = [
"The sound of waves crashing against the shore filled the air, as surfers rode the powerful swells with skill and grace, showcasing their love for the ocean.",
"Amidst the hustle and bustle of the city streets, a street performer mesmerized the crowd with his soulful melody, transporting them to a world of music and harmony.",
"In the heart of the forest, a group of hikers embarked on a journey through lush green trails, immersing themselves in the beauty of nature and discovering hidden treasures along the way."
]
# Підготовка нових документів
tokenized_new_documents = [word_tokenize(text) for text in new_documents]
new_text_bigrams = [bigram_model[doc] for doc in tokenized_new_documents]
new_bow_corpus = [dictionary.doc2bow(text) for text in new_text_bigrams]

# Отримання векторного представлення документів
new_document_vectors = lsi_bow[new_bow_corpus]

# Визначення тем для кожного документа
for i, doc_vector in enumerate(new_document_vectors):
    doc_topics = lsi_bow.get_document_topics(doc_vector)
    sorted_topics = sorted(doc_topics, key=itemgetter(1), reverse=True)
    top_topic = sorted_topics[0][0]  # Найбільш важлива тема для документа
    print(f"Документ {i+1} належить до теми #{top_topic+1}")


AttributeError: 'LsiModel' object has no attribute 'get_document_topics'

In [ ]:
new_documents = [
"The sound of waves crashing against the shore filled the air, as surfers rode the powerful swells with skill and grace, showcasing their love for the ocean.",
"Amidst the hustle and bustle of the city streets, a street performer mesmerized the crowd with his soulful melody, transporting them to a world of music and harmony.",
"In the heart of the forest, a group of hikers embarked on a journey through lush green trails, immersing themselves in the beauty of nature and discovering hidden treasures along the way."
]
# виконуємо поперідню обробку
preprocessed_new_documents = [preproc_doc(doc) for doc in new_documents]
# токенізуємо документи
tokenized_new_documents = [word_tokenize(doc) for doc in preprocessed_new_documents]
# виділяємо біграми
bigram = gensim.models.Phrases(tokenized_new_documents, min_count=20, threshold=20, delimiter='_')
bigram_model = gensim.models.phrases.Phraser(bigram)
text_bigrams_new_documents = [bigram_model[doc] for doc in tokenized_new_documents]
# створюємо словник
dictionary_new = gensim.corpora.Dictionary(text_bigrams_new_documents)
dictionary_new.filter_extremes(no_below=1, no_above=0.9)
# створюємо модель "Сумка слів"
bow_corpus_new_documents = [dictionary_new.doc2bow(text) for text in text_bigrams_new_documents]
# реалізовуємо модель прихованого семантичного індексування
Total_topics = 3
lsi_bow_new = gensim.models.LsiModel(bow_corpus_new_documents, id2word=dictionary_new, num_topics=Total_topics, onepass=True, chunksize=1740, power_iters=1000)
# виводимо основні теми
for topic_id, topic in lsi_bow_new.print_topics(num_topics=3, num_words=5):
    print('Topic #'+str(topic_id+1)+':')
    print(topic)

In [ ]:
# формуємо датафрейм для демонстрації приналежності кожного документу до певної теми
term_topic_new = lsi_bow_new.projection.u
singular_values_new = lsi_bow_new.projection.s
topic_document_new = (gensim.matutils.corpus2dense(lsi_bow_new[bow_corpus_new_documents], len(singular_values_new)).T / singular_values_new).T
document_topics = pd.DataFrame(np.round(topic_document_new.T, 3), columns=['T'+str(i) for i in range(1, Total_topics+1)])
print(document_topics)

In [ ]:
# знаходимо біграми використовуючи підхід групування.
# визначимо функцію для обчислення n-грам на основі деякого
# вхідного списку токенів і параметра n, який визначає ступінь n-грами.
def compute_ngrams(sequence, n):
    return list(zip(*(sequence[index:] for index in range(n))))
# визначаємо функцію для зведення корпусу в один великий рядок тексту
def flatten_corpus(corpus):
    return ' '.join([document.strip() for document in corpus])
# задаємо функцію для отримання найчастіших n-грам тексту
def get_top_ngrams(corpus, ngram_val=1, limit=5):
    corpus = flatten_corpus(corpus)
    tokens = nltk.word_tokenize(corpus)
    ngrams = compute_ngrams(tokens, ngram_val)
    ngrams_freq_dist = nltk.FreqDist(ngrams)
    sorted_ngrams_fd = sorted(ngrams_freq_dist.items(), key=itemgetter(1), reverse=True)
    sorted_ngrams = sorted_ngrams_fd[0:limit]
    sorted_ngrams = [(' '.join(text), freq) for text, freq in sorted_ngrams]
    return sorted_ngrams
# токенізуємо та пропускаємо через функцію попередньої обробки наш текст
text = nltk.word_tokenize(preproc_doc(gutenberg.raw('austen-persuasion.txt')))
# отримуємо десять найчастіших біграм з цього тексту
get_top_ngrams(corpus=text, ngram_val=2,limit=10)